In [1]:
import pandas as pd
import numpy as np
!pip install geopandas
import geopandas as gpd
import altair as alt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 3.9 MB/s 
     |████████████████████████████████| 16.7 MB 259 kB/s 
     |████████████████████████████████| 6.3 MB 45.2 MB/s 


In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
#Reading the file

fireData = pd.read_excel('https://brown-institute-assets.s3.amazonaws.com/Objects/pointsunknown/WildfireRiskToCommunities_County_Summary_2020-03-31.xlsx')

In [4]:
fireData

,STATEFP,STATE,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,Total number of Housing Units (HUs),Fraction of Total HUs Directly Exposed,Fraction of Total HUs Indirectly Exposed,Fraction of Total HUs Not Exposed,Total number of exposed HUs,Fraction of Exposed HUs Directly Exposed,Fraction of Exposed HUs Indirectly Exposed,Mean BP,Mean BP percentile within state,Mean BP percentile within nation,Mean BP-Direct,Mean BP-Indirect,Mean CRPS,Mean cRPS percentile within state,Mean cRPS percentile within nation,Mean CRPS-Direct,Mean CRPS-Indirect,Mean RPS,Mean RPS percentile within state,Mean RPS percentile within nation,Mean RPS-Direct,Mean RPS-Indirect,Mean WHP,Mean WHP percentile within state,Mean WHP percentile within nation,Mean WHP-Direct,Mean WHP-Indirect,Mean CFL,Mean CFL-Direct,Mean CFL-Indirect,Mean FLEP4,Mean FLEP4-Direct,Mean FLEP4-Indirect,Mean FLEP8,Mean FLEP8-Direct,Mean FLEP8-Indirect,Expected Annual HU Exposed,EA HUexp percentile within state,EA HUexp percentile within nation,Fraction EAHUexposed-Direct,Fraction EAHUexposed-Indirect,Expected Annual Relative HU Risk,EA HUrisk percentile within state,EA Hurisk percentile within nation,Fraction EAHUrisk-Direct,Fraction EAHUrisk-Indirect
0,1,Alabama,1,161526,1001,Autauga,Autauga County,21818.181820,0.596639,0.363112,0.040249,20940.010983,0.621660,0.378340,0.000112,0.333225,0.532134,0.000151,0.000047,37.264080,0.886254,0.366677,37.932941,36.165054,0.004249,0.471252,0.525618,0.005776,0.001741,185.637024,0.803761,0.701563,201.124939,160.188400,2.642163,2.725060,2.505953,0.393904,0.407219,0.372024,0.064566,0.072951,0.050787,2.343266,0.567164,0.632919,0.840768,0.159232,88.976905,0.582090,0.617001,0.844994,0.155006
1,1,Alabama,3,161527,1003,Baldwin,Baldwin County,82255.731230,0.625038,0.367117,0.007845,81610.399520,0.629980,0.370020,0.001012,0.917000,0.843931,0.001216,0.000666,35.674931,0.672400,0.262950,36.694511,33.939037,0.038371,0.996276,0.816062,0.046757,0.024093,184.031097,0.791907,0.695695,199.083115,158.404190,2.622066,2.762373,2.383183,0.279015,0.288208,0.263364,0.142082,0.140701,0.144434,82.606903,0.985075,0.971028,0.756631,0.243369,3131.475108,0.985075,0.959885,0.767669,0.232331
2,1,Alabama,5,161528,1005,Barbour,Barbour County,10190.513830,0.640346,0.313473,0.046181,9719.904260,0.671350,0.328650,0.000129,0.503230,0.561358,0.000155,0.000076,38.180351,0.925544,0.434433,39.331966,35.827892,0.005124,0.546554,0.545617,0.006203,0.002920,115.649498,0.427609,0.568413,115.053070,116.867859,2.786818,2.922653,2.509341,0.429082,0.439774,0.407239,0.082610,0.092423,0.062564,1.256677,0.283582,0.537090,0.805651,0.194349,49.804974,0.298507,0.520853,0.812702,0.187298
3,1,Alabama,7,161529,1007,Bibb,Bibb County,8903.952569,0.803326,0.196673,0.000000,8903.952436,0.803326,0.196673,0.000221,0.753708,0.620498,0.000236,0.000160,35.233353,0.619495,0.237200,35.589233,33.779736,0.007961,0.753708,0.603327,0.008596,0.005366,164.817200,0.614633,0.666550,169.177765,147.006149,2.219790,2.248582,2.102189,0.335944,0.340359,0.317910,0.057087,0.060346,0.043774,1.965698,0.507463,0.611270,0.857127,0.142873,70.884260,0.507463,0.581662,0.867435,0.132565
4,1,Alabama,9,161530,1009,Blount,Blount County,22784.584980,0.825594,0.174406,0.000000,22784.584640,0.825594,0.174406,0.000310,0.790347,0.661075,0.000330,0.000213,31.341494,0.111538,0.047377,31.248764,31.780451,0.009652,0.787353,0.618421,0.010299,0.006591,98.634033,0.361154,0.534771,99.393005,95.041260,1.770975,1.741407,1.910940,0.308756,0.312817,0.289532,0.045455,0.046810,0.039043,7.056905,0.805970,0.795606,0.880262,0.119738,219.915048,0.791045,0.744349,0.880906,0.119094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,56,Wyoming,37,1609192,56037,Sweetwater,Sweetwater County,17437.549410,0.275119,0.724226,0.000655,17426.127375,0.275300,0.724700,0.000114,0.103984,0.538686,0.000219,0.000074,45.900665

In [5]:
#Choosing relevant columnns to visiualize

exposedHousingUnits = fireData[['STATEFP','STATE','NAME','Total number of Housing Units (HUs)','Total number of exposed HUs']]
exposedHousingUnits.head()



,STATEFP,STATE,NAME,Total number of Housing Units (HUs),Total number of exposed HUs
0,1,Alabama,Autauga,21818.181820,20940.010983
1,1,Alabama,Baldwin,82255.731230,81610.399520
2,1,Alabama,Barbour,10190.513830,9719.904260
3,1,Alabama,Bibb,8903.952569,8903.952436
4,1,Alabama,Blount,22784.584980,22784.584640


In [ ]:
#Calculating the sums of total houses and number of exposed

stateHouses = exposedHousingUnits.groupby(['STATEFP','STATE'])[['Total number of Housing Units (HUs)',	'Total number of exposed HUs']].sum().reset_index()
stateHouses

In [7]:
#Calculating percent of houses exposed 
stateHouses['percent_exposed'] = stateHouses['Total number of exposed HUs'] / stateHouses['Total number of Housing Units (HUs)'] * 100

In [8]:
#Calculating ratio of houses exposed 

stateHouses['fraction_exposed'] = stateHouses['Total number of exposed HUs'] / stateHouses['Total number of Housing Units (HUs)']

In [ ]:
stateHouses

In [80]:
#Reading statepolygon file using geopandas
statesPolygons = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_state_500k.zip')

In [11]:
#converting statefp column into int
statesPolygons.STATEFP = statesPolygons.STATEFP.astype(int)

In [12]:
#Merging the two dataframes

mergedDF = statesPolygons.merge(stateHouses, on='STATEFP')

In [ ]:
#Filtering mergedDF of unwanted columns
mergedDF = mergedDF[['STATE','Total number of Housing Units (HUs)', 'Total number of exposed HUs',
       'percent_exposed', 'fraction_exposed','geometry']]
mergedDF.head()

In [ ]:
#Plotting the map using the newly created dataframe, mergedDF

alt.Chart(mergedDF).mark_geoshape(
    fill='lightgray',
    stroke='darkgray',
    strokeWidth=0.5
).properties(
    width=700,
    height=350
).project(
    type='albersUsa'
).configure_view(strokeWidth=0)

In [ ]:
#Plotting by percent of houses exposed to fire
statesMap = alt.Chart(mergedDF).mark_geoshape().encode(
    color=alt.Color('percent_exposed:Q',
      legend=alt.Legend(title='percent_exposed'),
      scale=alt.Scale(scheme='lightorange',
        )),
    tooltip=[alt.Tooltip('STATE',title='State'),
      alt.Tooltip('percent_exposed',title='% of housing units exposed', format='.2f')]
).project(
    type='albersUsa'
).properties(
    title='Direct Exposure to Wildfire Risk',
    width=700,
    height=450
).configure_view(
    strokeWidth=0
)

statesMap

In [ ]:
#Plotting by ratio of houses exposed to fire
statesMap = alt.Chart(mergedDF).mark_geoshape().encode(
    color=alt.Color('fraction_exposed:Q',
      legend=alt.Legend(title='fraction_exposed'),
      scale=alt.Scale(scheme='lightorange',
        )),
    tooltip=[alt.Tooltip('STATE',title='State'),
      alt.Tooltip('fraction_exposed',title='ratio of housing units exposed', format='.2f')]
).project(
    type='albersUsa'
).properties(
    title='Ratio of units Exposure to Wildfire Risk',
    width=700,
    height=450
).configure_view(
    strokeWidth=0
)

statesMap

In [ ]:
statesMap = alt.Chart(mergedDF).mark_geoshape().encode(
    color=alt.Color('percent_exposed:Q',
      bin=alt.Bin(maxbins=5, step=0),
      legend=alt.Legend(title='% of Housing Units',format='.2f'),
      scale=alt.Scale(scheme='lightorange')),
    tooltip=[alt.Tooltip('STATE',title='State'),
      alt.Tooltip('percent_exposed:Q',title='% of housing units exposed',format='.2f')]
).project(
    type='albersUsa'
).properties(
    title='Percent of exposed to Wildfire Risk',
    width=700,
    height=450
).configure_view(
    strokeWidth=0
)

statesMap

In [ ]:
alt.Chart(mergedDF).mark_bar(fill='#f8936d').encode(
    x=alt.X('percent_exposed:Q',
      title='Percentage of Housing Units',
      axis=alt.Axis(format='.0%'),
      bin=alt.Bin(maxbins=5,step=0.2)),
    y=alt.Y('percent_exposed',
      title='Number of Counties')
).properties(
    title='Number of Counties per Percentage of Housing Units with Direct Exposure to Fire Risk'
).configure_view(
    strokeWidth=0
)

In [19]:
mergedDF['quantiles'] = pd.qcut(mergedDF['fraction_exposed'], 5, labels=['0 - 20 percentile','20 - 40 percentile','40 - 60 percentile','60 - 80 percentile','80 - 100 percentile'])

In [ ]:


statesMap = alt.Chart(mergedDF).mark_geoshape().encode(
    color=alt.Color('quantiles:O',
      legend=alt.Legend(title='% of Housing Units'),
      scale=alt.Scale(scheme='lightorange')),
    tooltip=[alt.Tooltip('STATE',title='State'),
      alt.Tooltip('percent_exposed',title='% of housing units',format='.2f')]
).project(
    type='albersUsa'
).properties(
    title='Direct Exposure to Wildfire Risk',
    width=700,
    height=450
).configure_view(
    strokeWidth=0
)

statesMap

In [ ]:
mergedDF

In [22]:
print('Range of values for first quantile:',mergedDF[mergedDF.quantiles == '0 - 20 percentile']['fraction_exposed'].min(), '-', mergedDF[mergedDF.quantiles == '0 - 20 percentile']['fraction_exposed'].max())
print('Range of values for second quantile:',mergedDF[mergedDF.quantiles == '20 - 40 percentile']['fraction_exposed'].min(), '-', mergedDF[mergedDF.quantiles == '20 - 40 percentile']['fraction_exposed'].max())
print('Range of values for third quantile:',mergedDF[mergedDF.quantiles == '40 - 60 percentile']['fraction_exposed'].min(), '-', mergedDF[mergedDF.quantiles == '40 - 60 percentile']['fraction_exposed'].max())
print('Range of values for fourth quantile:',mergedDF[mergedDF.quantiles == '60 - 80 percentile']['fraction_exposed'].min(), '-', mergedDF[mergedDF.quantiles == '60 - 80 percentile']['fraction_exposed'].max())
print('Range of values for fifth quantile:',mergedDF[mergedDF.quantiles == '80 - 100 percentile']['fraction_exposed'].min(), '-', mergedDF[mergedDF.quantiles == '80 - 100 percentile']['fraction_exposed'].max())

Range of values for first quantile: 0.29153563828281126 - 0.5681811388217884
Range of values for second quantile: 0.5869562567392737 - 0.6188666195711573
Range of values for third quantile: 0.6234280475583039 - 0.7572253628360188
Range of values for fourth quantile: 0.7587909022552773 - 0.8999405183671711
Range of values for fifth quantile: 0.8999662309030465 - 0.9748489656617902


In [ ]:
alt.Chart(mergedDF).mark_bar(fill='#f8936d').encode(
    x=alt.X('fraction_exposed:Q',
      title='Ratio of Housing Units exposed by state',
      axis=alt.Axis(format='.0%'),
      bin=alt.Bin(maxbins=20)),
    y=alt.Y('count()',
      title='Number of States')
).properties(
    title='Ratio of housing units by states with Direct Exposure to Fire Risk'
).configure_view(
    strokeWidth=0
)

In [24]:
!pip install jenkspy
import jenkspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.2 MB/s 
  Created wheel for jenkspy: filename=jenkspy-0.2.0-cp37-cp37m-linux_x86_64.whl size=72617 sha256=50cb145d627e62d2aaca7ef5ad728c4ef715539b3882b5039a67b4ca94c70eba
  Stored in directory: /root/.cache/pip/wheels/d8/d1/42/9bd05164ef706b84674905892f0542f8db6ac4a652473ca7e0
Successfully built jenkspy


In [25]:
breaks = jenkspy.jenks_breaks(mergedDF['fraction_exposed'], nb_class=5)

In [26]:
breaks

[0.29153563828281126,
 0.4559967492996932,
 0.6264986557576298,
 0.7587909022552773,
 0.8811557615196294,
 0.9748489656617902]

In [27]:
mergedDF['naturalBreaks'] = pd.cut(mergedDF['fraction_exposed'], breaks, labels=naturalBreaksLabels)

NameError: ignored

In [28]:
mergedDF = mergedDF(['STATE', 'Total number of Housing Units (HUs)',
       'Total number of exposed HUs', 'percent_exposed', 'fraction_exposed',
       'geometry'])

TypeError: ignored

In [ ]:
mergedDF['naturalBreaks'].value_counts(ascending=True)

In [29]:
mergedDF.columns

Index(['STATE', 'Total number of Housing Units (HUs)',
       'Total number of exposed HUs', 'percent_exposed', 'fraction_exposed',
       'geometry', 'quantiles'],
      dtype='object')

In [30]:
gdf = gpd.GeoDataFrame(mergedDF, crs="EPSG:4326", geometry=mergedDF.geometry) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  """Entry point for launching an IPython kernel.


In [31]:
# my_geoseries = my_geoseries.set_crs("EPSG:4326")

In [ ]:
# gdf.to_file("fireGEO.geojson", driver='GeoJSON')

In [62]:
fireCost = pd.read_csv("state-cost-data.csv")

In [63]:
fireCost['total_spend'] = fireCost['drought']	+ fireCost['flooding']	+ fireCost['freeze']	+ fireCost['severe storm']	+ fireCost['tropical cyclone']	+ fireCost['wildfire']	+ fireCost['winter storm']

In [64]:
fireCost.head()

,state,drought,flooding,freeze,severe storm,tropical cyclone,wildfire,winter storm,total_spend
0,AK,0.0,0.0,0.0,0.0,0.0,2038.0,0.0,2038.0
1,AL,6181.9,124.0,136.9,11879.4,24289.8,661.5,2012.9,45286.4
2,AR,5977.3,4240.1,268.0,6110.1,648.8,0.0,873.3,18117.6
3,AZ,1051.1,603.0,0.0,4940.0,0.0,1181.9,0.0,7776.0
4,CA,12340.1,13029.6,13942.0,3172.8,0.0,88809.5,0.0,131294.0


In [ ]:
#Renaming STUPS to merge with cost dataframe
statesPolygons.rename(columns={"STUSPS":"state"})

In [82]:
statesPolygons = statesPolygons.rename(columns={"STUSPS":"state"})

In [83]:
mergedCost = statesPolygons.merge(fireCost, on='state')

In [85]:
mergedCost.head()

,STATEFP,STATENS,AFFGEOID,GEOID,state,NAME,LSAD,ALAND,AWATER,geometry,drought,flooding,freeze,severe storm,tropical cyclone,wildfire,winter storm,total_spend
0,56,01779807,0400000US56,56,WY,Wyoming,00,251458712294,1867503716,"POLYGON ((-111.05456 45.00095, -111.04507 45.0...",2489.2,0.0,7.5,950.4,0.0,994.9,0.0,4442.0
1,02,01785533,0400000US02,02,AK,Alaska,00,1478942847588,245378425142,"MULTIPOLYGON (((179.48246 51.98283, 179.48657 ...",0.0,0.0,0.0,0.0,0.0,2038.0,0.0,2038.0
2,24,01714934,0400000US24,24,MD,Maryland,00,25151992308,6979074857,"MULTIPOLYGON (((-76.05015 37.98691, -76.04998 ...",3049.2,246.1,30.2,2545.9,5796.3,0.0,2497.1,14164.8
3,05,00068085,0400000US05,05,AR,Arkansas,00,134660767709,3121950081,"POLYGON ((-94.61792 36.49941, -94.61765 36.499...",5977.3,4240.1,268.0,6110.1,648.8,0.0,873.3,18117.6
4,38,01779797,0400000US38,38,ND,North Dakota,00,178694310772,4414779956,"POLYGON ((-104.04868 48.86378, -104.04865 48.8...",21172.4,8782.9,11.3,74.4,0.0,11.9,34.9,30087.8


In [86]:
mergedCost['percent_wildfire'] = mergedCost['wildfire'] / mergedCost['total_spend'] * 100

In [ ]:
mergedCost.sort_values(by="percent_wildfire", ascending=False)

In [90]:
mergedCost.columns

Index(['STATEFP', 'STATENS', 'AFFGEOID', 'GEOID', 'state', 'NAME', 'LSAD',
       'ALAND', 'AWATER', 'geometry', 'drought', 'flooding', 'freeze',
       'severe storm', 'tropical cyclone', 'wildfire', 'winter storm',
       'total_spend', 'percent_wildfire'],
      dtype='object')

In [ ]:
#Plotting by percent of houses exposed to fire
statesMap = alt.Chart(mergedCost).mark_geoshape().encode(
    color=alt.Color('percent_wildfire:Q',
      legend=alt.Legend(title='percent_exposed'),
      scale=alt.Scale(scheme='lightorange',
        )),
    tooltip=[alt.Tooltip('state',title='State'),
      alt.Tooltip('percent_wildfire',title='% of housing units exposed', format='.2f')]
).project(
    type='albersUsa'
).properties(
    title='Direct Exposure to Wildfire Risk',
    width=700,
    height=450
).configure_view(
    strokeWidth=0
)

statesMap

In [ ]:
alt.Chart(mergedCost).mark_bar(fill='#f8936d').encode(
    x=alt.X('percent_wildfire:Q',
      title='Percentage of Housing Units',
      axis=alt.Axis(format='.0%'),
      bin=alt.Bin(maxbins=5,step=0.2)),
    y=alt.Y('count()',
      title='Number of Counties')
).properties(
    title='Number of Counties per Percentage of Housing Units with Direct Exposure to Fire Risk'
).configure_view(
    strokeWidth=0
)

In [ ]:
#Visualizing where cost of wildfires where a bigger proportion of overall cost of disasters

alt.Chart(mergedCost.sort_values(by='percent_wildfire', ascending=False).head(10)).mark_bar().encode(
    x=alt.X('state:N', sort='-y'),
    y="percent_wildfire:Q",
    # The highlight will be set on the result of a conditional statement
).properties(width=400)

In [ ]:
#Visualizing where cost of disasters where higher
alt.Chart(mergedCost.sort_values(by='total_spend', ascending=False).head(10)).mark_bar().encode(
    x=alt.X('state:N', sort='-y'),
    y="total_spend:Q",
    # The highlight will be set on the result of a conditional statement
).properties(width=400)


# alt.Chart(source).mark_bar().encode(
#     x='sum(yield):Q',
#     y=alt.Y('site:N', sort='-x')
# )